In [ ]:
batch_size=32

transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor()
])

dataset = [(transform(img), target) for img, target in dataset]

torch.manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(
   train_dataset, batch_size=batch_size, shuffle=True
)
test_loader = DataLoader(
   test_dataset, batch_size=batch_size, shuffle=False
)

NameError: name 'transforms' is not defined

In [ ]:
class CNN_detection(nn.Module):
  def __init__(self, image_size, num_objects, num_classes):
    super().__init__()

    self.conv_part = torch.nn.Sequential(
        torch.nn.Conv2d(3, 64, 3, padding=1), torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 2),
        torch.nn.Conv2d(64, 64, 3, padding=1), torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 2),
        torch.nn.Conv2d(64, 32, 3, padding=1), torch.nn.ReLU(),
        torch.nn.Conv2d(32, 8, 3, padding=1), torch.nn.ReLU(),
        torch.nn.Conv2d(8, 1, 3, padding=1), torch.nn.ReLU(),
        torch.nn.Flatten(),
    )
    self.lin_part = torch.nn.Sequential(
        torch.nn.Linear(12544, 128), torch.nn.ReLU(),
        torch.nn.Linear(128, 64), torch.nn.ReLU(),
        torch.nn.Linear(64, num_objects*(num_classes+4))
    )

  def forward(self, img):
      r = self.conv_part(img)
      out = self.lin_part(r)
      return out


In [ ]:
def extract_single_object(dataset_item, obj_index=0):
    image, target = dataset_item
    objs = target['annotation']['object']

    # Handle single-object edge case
    if not isinstance(objs, list):
        objs = [objs]

    obj = objs[obj_index]
    bbox = obj['bndbox']

    xmin = int(bbox['xmin'])
    ymin = int(bbox['ymin'])
    xmax = int(bbox['xmax'])
    ymax = int(bbox['ymax'])
    class_name = obj['name']

    return image, class_name, xmin, ymin, xmax, ymax


In [ ]:
# the dataset has 21 classes and at most 50 objects

model = CNN_detection(448, 50, 21)

loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train
for epoch in range(10):
    for batch in train_loader:
        image, class_name, xmin, ymin, xmax, ymax = extract_single_object(batch)
        print(class_name, xmin, ymin, xmax, ymax)
        image = transform(image)
        optimizer.zero_grad()
        output = model(batch)
        loss_value = loss(output, parse_target(class_name, xmin, ymin, xmax, ymax))
        loss_value.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss_value.item()}")

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>